In [73]:
# Import modules
import pandas as pd
import cleaning
import utils
import importlib
from cleaning import replace_error_values

In [74]:
# reload utils
importlib.reload(utils)

<module 'utils' from 'd:\\Users\\User\\Downloads\\RA Work\\acs\\notebooks\\utils.py'>

In [75]:
# open elections.xlsx
elections = pd.read_excel('../input/elections.xlsx')
# open elections_with_geocodes.xlsx
geocodes = pd.read_excel('../input/elections_with_geocodes.xlsx')
# open elections_with_tracts.xlsx
tracts = pd.read_excel('../input/elections_with_tracts.xlsx')
sv = pd.read_excel('../input/selected_variables.xlsx')

key = 'ad8851f3bf6aaf76923ec4119b6f714cdfaa87d9'

In [76]:
tracts_dict = {'geocode': [], 'year': []}

# for each row in elections
for index, row in elections.iterrows():
    # if District # is not nan
    if not pd.isna(row['District #']):
        # get row in tracts where filename is the same
        tract_row = tracts[tracts['filename'] == row['filename']]
        # make sure is not nan
        if not pd.isna(tract_row['tracts'].iloc[0]):
            new_tracts = tract_row['tracts'].iloc[0].split(', ')
            # add to tracts_df
            for new_tract in new_tracts:
                tracts_dict['geocode'].append(new_tract)
                tracts_dict['year'].append(row['year'])
            
# create tracts_df
tracts_df = pd.DataFrame(tracts_dict)

# remove duplicates
tracts_df = tracts_df.drop_duplicates()

# reset index on tracts_df
tracts_df = tracts_df.reset_index(drop=True)

In [77]:
acs_data = []

In [78]:
# for each row in tracts_df
for index, row in tracts_df.iterrows():
    if index < len(acs_data):
        continue

    if index % 10 == 0:
        print(f"{index} of {len(tracts_df)}")

    geocode = row['geocode'].split('_')
    state = geocode[0].zfill(2)
    county = geocode[1].zfill(3)
    tract = geocode[2].zfill(6)
    year = row['year']
    
    for_param = f'tract:{tract}&in=state:{state}+county:{county}'
    data_dict = utils.query(year, sv, for_param, key)
    data_dict['geocode'] = row['geocode']
    data_dict['year'] = year

    acs_data.append(data_dict)

0 of 6410
10 of 6410
20 of 6410
30 of 6410
40 of 6410
50 of 6410
60 of 6410
70 of 6410
80 of 6410
90 of 6410
100 of 6410
110 of 6410
120 of 6410
130 of 6410
140 of 6410
150 of 6410
160 of 6410
170 of 6410
180 of 6410
190 of 6410
200 of 6410
210 of 6410
220 of 6410
230 of 6410
240 of 6410
250 of 6410
260 of 6410
270 of 6410
280 of 6410
290 of 6410
300 of 6410
310 of 6410
320 of 6410
330 of 6410
340 of 6410
350 of 6410
360 of 6410
370 of 6410
380 of 6410
390 of 6410
400 of 6410
410 of 6410
420 of 6410
430 of 6410
440 of 6410
450 of 6410
460 of 6410
470 of 6410
480 of 6410
490 of 6410
500 of 6410
510 of 6410
520 of 6410
530 of 6410
540 of 6410
550 of 6410
560 of 6410
570 of 6410
580 of 6410
590 of 6410
600 of 6410
610 of 6410
620 of 6410
630 of 6410
640 of 6410
650 of 6410
660 of 6410
670 of 6410
680 of 6410
690 of 6410
700 of 6410
710 of 6410
720 of 6410
730 of 6410
740 of 6410
750 of 6410
760 of 6410
770 of 6410
780 of 6410
790 of 6410
800 of 6410
810 of 6410
820 of 6410
830 of 6410
840

In [81]:
df = cleaning.dicts_to_df(acs_data)

# order so that geocode and year are first
df = df[['geocode', 'year'] + [col for col in df.columns if col not in ['geocode', 'year']]]
replace_error_values(df)

# drop state, county, and tract
df = df.drop(columns=['state', 'county', 'tract'])

# save as csv
df.to_csv('../output/tract_acs_data.csv', index=False)